In [19]:
import albumentations as A
import cv2
import os
import numpy as np
from tqdm import tqdm
import shutil

In [26]:
def get_augmentation():
    return A.Compose([
        A.RandomRotate90(p=0.75),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Transpose(p=0.75),
        A.OneOf([
            A.GridDistortion(p=0.5),
            A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=0.5),
        ], p=0.9),
        A.OneOf([
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        ], p=0.9),
        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
        ], p=0.4),
        A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),
    ])

In [27]:
def resize_mask(mask, target_size):
    return cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)

In [28]:
def augment_and_save(image_path, mask_path, output_image_path, output_mask_path, augmentation):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    mask = resize_mask(mask, (image.shape[1], image.shape[0]))

    augmented = augmentation(image=image, mask=mask)
    augmented_image = augmented['image']
    augmented_mask = augmented['mask']

    cv2.imwrite(output_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
    cv2.imwrite(output_mask_path, augmented_mask)

In [29]:
def augment_dataset(input_dir, output_dir, augmentations_per_image=10):
    augmentation = get_augmentation()

    input_images_dir = os.path.join(input_dir, 'images')
    input_masks_dir = os.path.join(input_dir, 'masks')

    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'masks'), exist_ok=True)

    print(f"Checking input images directory: {input_images_dir}")
    if not os.path.exists(input_images_dir):
        print(f"Error: Input images directory {input_images_dir} does not exist.")
        return

    image_files = [f for f in os.listdir(input_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(image_files)} image files in the input directory.")

    if not image_files:
        print("No image files found in the input directory.")
        return
    
    for image_file in tqdm(image_files, desc="Augmenting images"):
        image_path = os.path.join(input_images_dir, image_file)
        mask_file = f"({image_file.split('(')[1].split(')')[0]}) annotated.PNG"
        mask_path = os.path.join(input_masks_dir, mask_file)
        
        print(f"Processing image: {image_file}")
        print(f"Looking for mask: {mask_file}")
        
        if not os.path.exists(mask_path):
            print(f"Warning: Mask not found for {image_file}. Skipping this image.")
            continue

        shutil.copy(image_path, os.path.join(output_dir, 'images', image_file))
        shutil.copy(mask_path, os.path.join(output_dir, 'masks', mask_file))
        
        for i in range(augmentations_per_image):
            output_image_path = os.path.join(output_dir, 'images', f"{image_file.split('.')[0]}_aug_{i}.jpg")
            output_mask_path = os.path.join(output_dir, 'masks', f"{mask_file.split('.')[0]}_aug_{i}.PNG")
            augment_and_save(image_path, mask_path, output_image_path, output_mask_path, augmentation)

    print(f"Augmentation complete. Check the '{output_dir}' folder for results.")

In [30]:
input_dir = "/home/_sergei/DataspellProjects/vein-segmentation/data/22/dataset/train"
output_dir = "/home/_sergei/DataspellProjects/vein-segmentation/data/22/augmented_data"
augment_dataset(input_dir, output_dir, augmentations_per_image=10)

Checking input images directory: /home/_sergei/DataspellProjects/vein-segmentation/data/22/dataset/train/images
Found 32 image files in the input directory.


Augmenting images:   0%|          | 0/32 [00:00<?, ?it/s]

Processing image: photo no checkersboard (40).jpg
Looking for mask: (40) annotated.PNG


Augmenting images:   3%|▎         | 1/32 [00:00<00:19,  1.60it/s]

Processing image: photo no checkersboard (35).jpg
Looking for mask: (35) annotated.PNG


...

Processing image: photo no checkersboard (25).jpg
Looking for mask: (25) annotated.PNG


Augmenting images:  97%|█████████▋| 31/32 [00:15<00:00,  1.79it/s]

Processing image: photo no checkersboard (30).jpg
Looking for mask: (30) annotated.PNG


Augmenting images: 100%|██████████| 32/32 [00:15<00:00,  2.03it/s]

Augmentation complete. Check the '/home/_sergei/DataspellProjects/vein-segmentation/data/22/augmented_data' folder for results.
